*Name:* Thresa Kelly

*Professor:* Dr. Allison Kirkpatrick

*Date:* 12/2/21

*Science Goal:* Which galaxies in the EGS field are likely AGN, and what are their physical properties?

*Details:* CEERS is an approved project on JWST to observe the Extended Groth Strip with the NIRCam, NIRSpec, and MIRI instruments. Thresa is going to identify AGN in the MIRI field of view. She will plot their spectral energy distributions, look for spectra in the 3D-HST database, look for radio observations through the FIRST database, and make images.



In [6]:
# imports
from astropy.io import ascii

In [8]:
# read data from file 
data = ascii.read("EGS_for_Thresa.csv")

In [ ]:
# view data
print(data.columns, '\n')
print(data)

<TableColumns names=('ID','RA','Dec','Redshift','u','u Unc','g','g Unc','r','r Unc','i','i Unc','z','z Unc','y','y Unc','IRAC Ch 1','Ch 1 Unc','IRAC Ch 2','Ch 2 Unc','IRAC Ch 3','Ch 3 Unc','IRAC Ch 4','Ch 4 Unc','PACS 100','100 Unc','PACS 160','160 Unc','SPIRE 250','250 Unc','SPIRE 350','350 Unc','SPIRE 500','500 Unc')> 

             ID                 RA        Dec    ... SPIRE 500  500 Unc 
--------------------------- ---------- --------- ... --------- ---------
HELP_J141411.189+520025.636 213.546622 52.007121 ...       NaN       NaN
HELP_J141411.417+515923.419 213.547572 51.989839 ...       NaN       NaN
HELP_J141411.539+515959.135 213.548077  51.99976 ...       NaN       NaN
HELP_J141411.902+515927.909 213.549592 51.991086 ...       NaN       NaN
HELP_J141412.159+515941.025 213.550661 51.994729 ...       NaN       NaN
HELP_J141412.263+515904.194 213.551097 51.984498 ...       NaN       NaN
HELP_J141412.597+520041.294 213.552488 52.011471 ...  7254.822   5038.43
HELP_J141412.781+51